In [21]:
import importlib
import argparse
import os
import sys
import time
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

import random
import numpy as np
import yaml
from box import Box
from pprint import pprint
import wandb
from tqdm import tqdm
from datetime import timedelta
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

CONFIG_FILENAME = '/home/liranc6/ecg_forecasting/liran_project/mrdiff/src/config_ecg.yml'

assert CONFIG_FILENAME.endswith('.yml')

with open(CONFIG_FILENAME, 'r') as file:
    config = yaml.safe_load(file)

# Add the parent directory to the sys.path
ProjectPath = config['project_path']
sys.path.append(ProjectPath)

import liran_project
from liran_project.mrdiff.src.parser import parse_args
from liran_project.utils.dataset_loader import SingleLeadECGDatasetCrops_mrDiff as DataSet
from liran_project.utils.util import ecg_signal_difference
from liran_project.mrdiff.exp_main import Exp_Main
from liran_project.utils.common import *

# Add the directory containing the exp module to the sys.path
exp_module_path = os.path.join(ProjectPath, 'mrDiff')
sys.path.append(exp_module_path)

# from mrDiff.exp.exp_main import Exp_Main
from mrDiff.data_process.etth_dataloader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Wind, Dataset_Caiso, Dataset_Production, Dataset_Caiso_M, Dataset_Production_M
from mrDiff.data_process.financial_dataloader import DatasetH
from mrDiff.data_process.forecast_dataloader import ForecastDataset
from mrDiff.exp.exp_basic import Exp_Basic
from mrDiff.models_diffusion import DDPM
from mrDiff.utils.tools import EarlyStopping, adjust_learning_rate, visual
from mrDiff.utils.metrics import metric

from liran_project.mrdiff.src.parser import Args

In [22]:
args = Args(CONFIG_FILENAME)

# Now you can use args as needed
vars(args)

{'configs_filename': '/home/liranc6/ecg_forecasting/liran_project/mrdiff/src/config_ecg.yml',
 'configs': Box({'project_path': '/home/liranc6/ecg_forecasting', 'tqdm': 'terminal', 'resume_exp': {'resume': False, 'resume_from': False, 'resume_optimizer': True, 'resume_epoch': 'None', 'loss_and_metrics': True, 'resume_scheduler': True, 'resume_configuration': True, 'specific_chpt_path': '/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl10_ll105_pl30_lr0.001_bs8_invFalse_itr0/09_10_2024_1410/best_checkpoint.pth', 'was_resumed': False}, 'wandb': {'entity': 'liranc6', 'mode': 'online', 'project': 'mrdiff', 'resume': 'None', 'run_name': 'None', 'id': 'None', 'save_code': True, 'resume_from': 'None'}, 'general': {'random_seed': 42, 'evaluate': False, 'tag': None, 'dataset': 'icentia11k', 'features': 'S', 'training_mode': 'ONE', 'interval': 1000}, 'optimization': {'learning_rate': 0.001, 'batch_size': 8, 'test_batch_size': 8, 'patience': 10, 'weight_d

In [23]:
exp = Exp_Main(args)

Use GPU: cuda:0


In [24]:
results_path = \
    "/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr0/09_10_2024_1612/best_checkp_dtw_dist.pth"

In [25]:
exp.args.resume_exp

Box({'resume': False, 'resume_from': False, 'resume_optimizer': True, 'resume_epoch': 'None', 'loss_and_metrics': True, 'resume_scheduler': True, 'resume_configuration': True, 'specific_chpt_path': '/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl10_ll105_pl30_lr0.001_bs8_invFalse_itr0/09_10_2024_1410/best_checkpoint.pth', 'was_resumed': False})

In [26]:
# Reload the module
importlib.reload(liran_project.mrdiff.exp_main)

# Assuming `exp` is an existing instance of `Exp_Main`
exp.__class__ = liran_project.mrdiff.exp_main.Exp_Main

torch.cuda.empty_cache()
! gpustat

1373.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


nlp-2080-1                     Wed Oct  9 21:18:33 2024  535.146.02
[0] NVIDIA GeForce RTX 2080 Ti | 30°C,   0 % |   518 / 11264 MB | liranc6(262M)


In [27]:
resume_configs = exp.args.resume_exp
resume_configs['resume'] = True
resume_configs['resume_configuration'] = True
resume_configs['specific_chpt_path'] = results_path

exp.args.update_config_from_dict(resume_configs)


In [28]:
iteration = exp.load_checkpoint(results_path)
iteration

Successfully loaded model from specific_chpt_path


7

In [ ]:
exp.args.testing.patients.end_patient = 1

In [29]:
# Convert Box object to dictionary
config_dict = exp.args.configs

# Access the configuration values using dictionary syntax
random_seed = config_dict['general']['random_seed']
tag = config_dict['general']['tag']
dataset = config_dict['general']['dataset']
features = config_dict['general']['features']

learning_rate = config_dict['optimization']['learning_rate']
batch_size = config_dict['optimization']['batch_size']

context_len = config_dict['training']['sequence']['context_len']
label_len = config_dict['training']['sequence']['label_len']
model = config_dict['training']['model_info']['model']
pred_len = config_dict['training']['sequence']['pred_len']
iterations = config_dict['training']['iterations']['itr']

inverse = config_dict['data']['inverse']

setting = f"{model}_{dataset}_ft{features}_sl{context_len}_ll{label_len}_pl{pred_len}_lr{learning_rate}_bs{batch_size}_inv{inverse}_itr{iteration}"
setting

'DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr7'

In [30]:
fix_seed = random_seed
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [36]:
exp.datasets, exp.dataloaders = {}, {}
exp.args.testing.patients.end_patient = 1

In [37]:
exp.test(setting=setting,
         time_path='09_10_2024_1612',
         test = True,
         visualize = True,
         )

loading model


test_pbar:   0%|          | 0/12 [00:00<?, ?it/s]

/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr0/09_10_2024_1612/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr7/09_10_2024_2140/0.png


test_pbar:   8%|▊         | 1/12 [01:19<14:34, 79.52s/it, time_elapsed=9940.84 ms, batch=8, Steps=12]

/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr0/09_10_2024_1612/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr7/09_10_2024_2140/1.png


test_pbar:  17%|█▋        | 2/12 [02:41<13:28, 80.81s/it, time_elapsed=20154.93 ms, batch=8, Steps=12]

/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/mrDiff/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr0/09_10_2024_1612/DDPM_icentia11k_ftS_sl10_ll105000_pl30000_lr0.001_bs8_invFalse_itr7/09_10_2024_2140/2.png


test_pbar:  17%|█▋        | 2/12 [03:34<17:51, 107.17s/it, time_elapsed=20154.93 ms, batch=8, Steps=12]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/liranc6/miniconda3/envs/ecg/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1779706/381972392.py", line 1, in <module>
    exp.test(setting=setting,
  File "/home/liranc6/ecg_forecasting/liran_project/mrdiff/exp_main.py", line 557, in test
    results.append_ecg_signal_difference(true, pred, self.args.data.fs)
  File "/home/liranc6/ecg_forecasting/liran_project/mrdiff/exp_main.py", line 723, in append_ecg_signal_difference
  File "/home/liranc6/ecg_forecasting/liran_project/utils/util.py", line 328, in ecg_signal_difference
    _, info = nk.ecg_process(ecg_pred, sampling_rate=sampling_rate)
  File "/home/liranc6/miniconda3/envs/ecg/lib/python3.10/site-packages/neurokit2/ecg/ecg_process.py", line 126, in ecg_process
    delineate_signal, delineate_info = ecg_delineate(
  File "/home/liranc6/miniconda3/envs/ecg/lib/python3.10